# Collaborative Filtering for Implicit Feedback Datasets

### Cost function for squared error with regularization

Across all x (users) and y (items), find the values of u and i that minimize the summation below:

$\underset{x,y}min\underset{u,i}\sum 
c_{ui} (p_{ui} - x_u^Ty_i)^2 + \lambda
(\underset u \sum \parallel x_u \parallel ^2
+\underset u \sum \parallel y_i \parallel ^2)$

##### Where:

$x_u$ is user vector,
$y_i$ is item vector.

$p_{ui} = 1$ if interaction, 
$p_{ui} = 0$ if no interaction.

$c_{ui} = 1 + \alpha * r_{ui}$, where
$r_{ui}$ = # of interactions for a user-item pair, and $\alpha$ determines our confidence levels.

$\lambda$ is regularization term.

#### Explanation of cost function

We take the squared error of our prediction and 
multiply by the confidence, and regularize our $x$ and $y$ vectors with $\lambda$ to penalize overfitting. (larger values or smaller values?)

$\alpha$ allows us to influence our confidence levels. Clearly, our confidence increases when a producer samples the same artist multiple times, but by how much? $\alpha$ determines how important multiple samples are.

We add 1 so that non-interactions are not lost during the cost calculation.

### ALS Algorithm

However, we can't use the cost function above because of the size of the dataset. (m * n terms)

Therefore we modify the cost function to Alternating Least Squares, which works by holding either user vectors or item vectors constant and calculating the global minimum, then alternating to the other vector.

#### Compute user factors

$x_u = (Y^T C^u Y + \lambda I)^{-1}  Y^T C^u p(u)$

##### Where:

$Y$ is $n * f$ matrix of item-factors. 

$C^u$ is a $n*n$ diagonal matrix for user $u$ where $C^u_{ii} = c_{ui}$. This is our confidence matrix for n items.

$p(u)$ is vector of preferences for user $u$.


#### Recompute item factors

$y_i = (X^TC^iX + \lambda I)^-1 X^TC^ip(i)$

##### Where:
$X$ = $m * f$ matrix  of user_factors. 

$C^i$ is $m * m$ diagonal matrix for each item $i$ where $C_{uu}^i = c_{ui}$

$p(i)$ is vector of preferences for item $i$.

### Explaining recommendations

If $\hat{p}_{ui}$, the predicted preference of user $u$ at item $i$, is equal to $y_i^Tx_u$, we can substite our user_factor equation for $x_u$. This gives us:

$\hat{p}_{ui} =  y_i^T(Y^T C^u Y + \lambda I)^{-1}  Y^T C^u p(u)$

Denote $f*f$ matrix $(Y^T C^u Y + \lambda I)^{-1}$ as $W^u$

$W^u$ is considered the weight for user $u$


### Ranking Algorithm

$\overline{rank} = \frac{\sum_{u,i} r^t_{ui} * rank_{ui}}{\sum_{u,i} r^t_{ui}}$

#### where:
$r^t_{ui}$ is the # of interactions for observations in the test set, and 

$rank_{ui}$ are the percentile ranking of each item for each user.

#### Explanation

We can see that $\sum_{u,i} r^t_{ui}$ is in both the numerator and the denominator. If $rank_{ui}$ was not in the numerator, $\overline{rank}$ would simply equal 1. $rank_{ui}$ is the percentile ranking of each item for each user, such that the item most highly recommended has a $rank_{ui}$ of 0.00\% and the item least recommended has a $rank_{ui}$ of 100.00\%.

Therefore, if the algorithm is correct, the low percentages will cancel out the high $r^t_{ui}$, making the $\overline{rank}$ go towards 0.


They log_scaled their data, which makes sense.

# Implement Ranking Algorithm

First, train the model.

In [3]:
from pymongo import MongoClient
client = MongoClient()
db = client.whosampled
import numpy as np
import pandas as pd

import implicit
import matplotlib.pyplot as plt
plt.style.use('ggplot')

import scipy.sparse as sparse
from scipy.sparse import csr_matrix

import os, sys
os.environ["OPENBLAS_NUM_THREADS"]="1"

import random

from src.turn_db_main_into_utility_matrix import from_mongo_collection_to_utility_matrix

# Read in the data from the Mongo collection

song_prod, artist_prod, df = from_mongo_collection_to_utility_matrix(db.main_redo)

In [14]:
# Using the parameters I found to be best from the Grid Search
alpha = 20
factors = 5
iterations = 10

model = implicit.als.AlternatingLeastSquares(factors=100,iterations=15)

# train the model on a sparse matrix of item/user/confidence weights
sparse_artist_prod = csr_matrix(artist_prod)
model.fit(sparse_artist_prod)

# recommend items for a user
sparse_prod_artist = sparse_artist_prod.T.tocsr()
recommendations = model.recommend(0, sparse_prod_artist, 20, False)

# find related items
# related = model.similar_items(0, 20)

# sim_users = model.similar_users(0, 10)

100%|██████████| 15.0/15 [00:07<00:00,  2.87it/s]


In [160]:
def create_rank_ui_from_model(model):
    
    '''
    Takes a fitted model and creates the rank_ui element of the rank
    evaluation algorithm. These are every item's ranking of importance 
    for each user, with ranking as percentages. 
    '''
    
    item_vecs = model.item_factors
    user_vecs = model.user_factors

    #predictions is items, user
    predictions = item_vecs.dot(user_vecs.T)

    #We need it as u,i
    preds_prods_artists = predictions.T
    
    #get the rank of each item for each user
    order = np.flip(preds_prods_artists.argsort(axis = 1), axis = 1)
    ranks = order.argsort(axis = 1)

    #turn ranks into percentages
    percentages = (ranks / ranks.shape[0]) * 100
    return percentages

def return_rank_score(percentages, r_ui):
    
    '''
    Requires r_ui to be in form users, items.
    '''
    
    numerator = r_ui * percentages

    numer_summation = np.sum(np.sum(numerator))

    denominator = np.sum(np.sum(prod_artists))

    rank = numer_summation /denominator
    return rank
    
percentages = create_rank_ui_from_model(model)
rank = return_rank_score(percentages, artist_prod.T)
rank

1.1566672280043242

In [145]:
# What if you just recommend the most popular?

# Get most popular items in the mat
samples_per_artist = artist_prod.sum(axis = 1)
order = np.flip(samples_per_artist.values.argsort(), axis = 0)
ranks = order.argsort()

percentages = (ranks / len(samples_per_artist)) * 100

#turn to series so as to concatenate into DF
percentages = pd.Series(percentages)

populars = pd.concat([percentages] * 11284, axis= 1).T

(11284, 8203)


In [162]:
rank_pop = return_rank_score(populars, artist_prod.T)

In [163]:
rank_pop

0.0

### Test on hidden 1-10%

The ranking/ evaluation method for the above is to do this:

1. Randomly take 10% of all of the interactions. Save their position in the matrix and values. Sum them. This is $\sum_{u,i} r^t_{ui}$, for  $\overline{rank} = \frac{\sum_{u,i} r^t_{ui} * rank_{ui}}{\sum_{u,i} r^t_{ui}}$  

2. Replace them with zeros. 

3. Train your model on this dataframe.

4. We need the $rank_{ui}$ of our model for all $u, i$ in $r^t_{ui}$. Use this to compute rank.

5. Get the $rank_{ui}$ of popularity for all $u, i$ in $r^t_{ui}$. This is popularity rank score.

# Split the data into 2016 after and before.


In [4]:
# What percentage of the data is 2018, 2017, etc?

np.cumsum(df.groupby('new_song_year').count()['URL'].apply(
    lambda x: (x / len(df)) * 100 ).sort_index(
    ascending = False))

# If we take the data past 2016-2019 as our test set, we get about 5%.

new_song_year
2019      0.127584
2018      1.752243
2017      3.614425
2016      5.619121
2015      8.069004
2014     10.856848
2013     14.537780
2012     18.091127
2011     21.231863
2010     24.333238
2009     26.670467
2008     29.158353
2007     31.654383
2006     34.037759
2005     36.783528
2004     39.245626
2003     41.907244
2002     44.100601
2001     46.638707
2000     49.208030
1999     52.375911
1998     55.539721
1997     58.646525
1996     62.246020
1995     65.739647
1994     70.241731
1993     74.872755
1992     80.081979
1991     84.847374
1990     88.833693
           ...    
1982     98.813741
1981     98.892463
1980     98.946754
1979     99.013261
1978     99.082482
1977     99.134058
1976     99.201922
1975     99.280644
1974     99.363438
1973     99.455732
1972     99.523596
1971     99.611819
1970     99.686469
1969     99.771978
1968     99.857486
1967     99.892775
1966     99.929422
1965     99.940280
1964     99.944352
1963     99.951138
1962     99.96335

In [280]:
test = df[df.new_song_year > 2014]
train = df[df.new_song_year < 2015]
print(test.shape)
print(train.shape)

(5945, 18)
(67732, 18)


In [297]:
# Songs and users have to be in both, right?

#get intersection of sampled_artist and song_producer from train and test set
def get_intersection_of_train_and_test_set(train, test, column):
    
    '''
    Takes train and test set and a column, and returns both filtered
    to have values they both share
    '''
    test_column = set(test[column])
    train_column = set(train[column])
    both_columns = test_column.intersection(train_column)
    print([len(test_column), len(train_column), len(both_columns)])
    
    test = test[test[column].isin(list(both_columns))]

    train = train[train[column].isin(list(both_columns))]
    
    return train, test

train, test = get_intersection_of_train_and_test_set(
    train, test, "sampled_artist")
train, test = get_intersection_of_train_and_test_set(
    train, test, "new_song_producer")

#I needed to run this 3 times to work

#I have 1163 artists, and 544 producers shared. 

[1163, 1164, 1163]
[544, 544, 544]


In [302]:
artist_prod_test = pd.crosstab(test_filtered.sampled_artist, columns=test_filtered.new_song_producer )


In [305]:
artist_prod_train.shape

(1381, 8653)

In [303]:
artist_prod_train = pd.crosstab(train_filtered.sampled_artist, columns=train_filtered.new_song_producer)


In [306]:
# Using the parameters I found to be best from the Grid Search
alpha = 20
factors = 5
iterations = 10

model = implicit.als.AlternatingLeastSquares(factors=200,iterations=15)

# training data for the model.
sparse_artist_prod = csr_matrix(artist_prod_train)
model.fit(sparse_artist_prod)

# recommend items for a user
# sparse_prod_artist = sparse_artist_prod.T.tocsr()
# recommendations = model.recommend(0, sparse_prod_artist, 20, False)

# find related items
# related = model.similar_items(0, 20)

# sim_users = model.similar_users(0, 10)

100%|██████████| 15.0/15 [00:03<00:00,  4.76it/s]


In [307]:
percentages = create_rank_ui_from_model(model)

In [310]:
rank = return_rank_score(percentages, artist_prod_test.T)

ValueError: Unable to coerce to DataFrame, shape must be (1456, 1381): given (8653, 1381)